In [1]:
# !pip install geoplotlib
# !pip install folium --user
# !pip install geopandas --user

In [31]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pandabase

## Download relevant files

In [32]:
with open('df_complete.pickle','rb') as read_file:
    df_complete = pickle.load(read_file)
    
with open('cuisine_topic_info.pickle','rb') as read_file:
    cuisine_topic_info = pickle.load(read_file)
    
with open('dish_topic_info.pickle','rb') as read_file:
    dish_topic_info = pickle.load(read_file)

## Curating information for Topic Naming

Now that we have clustered the data into 30 restaurant profiles, I am going to name them using the same information we fed into our K-means model

**Cuisines:** Remember the LDA topic modeling earlier? The cuisine topic-keyword distribution and the document-topic distribution became useful in analyzing each clusters. I am going to get each cluster's top 3 Cuisine dimensions by their key topic keywords and percentage contribution to the final k-means clustering.\
**Dishes:** The same process as cuisines but with dishes LDA topic modeling\
**Price:** Get the average max price for each cluster\
**Menu Variability:** Get the average count of top dishes for each cluster

**Getting cuisine representation**

In [23]:
km_label_grped = df_complete.groupby(['km_label','cuisine_dominant_topic']).count()
tot = df_complete.groupby(['km_label']).count().reset_index()

# km_label_grped
g = km_label_grped['row_number'].groupby(level=0, group_keys=False)
res = g.apply(lambda x: x.sort_values(ascending=False).head(3)).reset_index()
merged = pd.merge(res, tot, how='left',on='km_label')
merged['perc'] = merged['row_number_x']/merged['row_number_y']

cuisine_inf = pd.merge(merged[['km_label','cuisine_dominant_topic_x','row_number_x','perc']],cuisine_topic_info, how='left',
        left_on='cuisine_dominant_topic_x', right_on = 'cuisine_dominant_topic')

cuisine_inf = cuisine_inf[['km_label','cuisine_dominant_topic_x','row_number_x','perc','top_doc_cuisines','cuisine_topic_keywords']]
cuisine_inf.columns = ['km_label','cuisine_dominant_topic','cuisine_topic_count','cuisine_topic_perc','top_doc_cuisines',
                       'cuisine_topic_keywords']

**Getting dish representation**

In [24]:
km_label_grped = df_complete.groupby(['km_label','dish_dominant_topic']).count()
tot = df_complete.groupby(['km_label']).count().reset_index()

# km_label_grped
g = km_label_grped['row_number'].groupby(level=0, group_keys=False)
res = g.apply(lambda x: x.sort_values(ascending=False).head(3)).reset_index()
merged = pd.merge(res, tot, how='left',on='km_label')
merged['perc'] = merged['row_number_x']/merged['row_number_y']

dish_inf = pd.merge(merged[['km_label','dish_dominant_topic_x','row_number_x','perc']],dish_topic_info, how='left',
        left_on='dish_dominant_topic_x', right_on = 'dish_dominant_topic')
dish_inf = dish_inf[['km_label','dish_dominant_topic_x','row_number_x','perc','top_doc_dish','dish_topic_keywords']]
dish_inf.columns = ['km_label1','dish_dominant_topic','dish_topic_count','dish_topic_perc','top_doc_dish',
                       'dish_topic_keywords']

**Getting price inference**

In [25]:
restaurant_inf = df_complete.groupby('km_label').agg({'row_number':'count','price_rating_x':'median','max_price':'median',
                                     'count_top_dishes':'median','unique_dishes':'median',
                                     'unique_hashtag_count':'median'}).reset_index()

**Joining Cuisine and Dish Interpretation into one table**

In [39]:
inference_df = pd.merge(pd.concat([cuisine_inf, dish_inf], axis=1), restaurant_inf, how = 'left', on = 'km_label')

## saving inference into excel
inference_df.to_excel("inference2.xlsx")